# Module 2.1 - Load Data from Database Using SQL

Many organizations still use relational database management systems (RDBMS) to store data. A relational database is an application that has a collection of tables, and each table has a row-column structure (similar to a spreadsheet). Databases use Structured Query Language (SQL) to do CRUD operations ("Create", "Read", "Update", or "Delete") on the information in a table (called "querying"). Each table in the database has a name, and the columns within a table are called fields. In this lesson, we will learn how to extract data by passing SQL statements to the database to return the information requested.

In [11]:
#library to use dataframes
import pandas as pd

#library to connect & interact with databases
from sqlalchemy import create_engine, inspect

### Connect to Database

An engine connects to the database and allows queries to be passed in and for information to be read from it. Each database application has its own engine statement; other examples can be found in the [SQL Alchemy documentation](https://docs.sqlalchemy.org/en/13/core/engines.html).

In [19]:
#set the database file location to a variable
db_file = r'datasets/boardgames.sqlite'

# set the connection to SQLite database in a variable
engine = create_engine(f"sqlite:///{db_file}")

In [20]:
#get a list of all the tables in the database
engine.table_names()

['BoardGames',
 'bgg.ldaOut.top.documents',
 'bgg.ldaOut.top.terms',
 'bgg.ldaOut.topics',
 'bgg.topics']

In [23]:
#check if a table exists
engine.has_table('bgg.topics')

True

In [24]:
#this is used to look at the schema of elements in a database
inspector = inspect(engine)

In [26]:
# get the fields (columns) and their attributes for the table called "test"
#this is a list where each item is a field(column)
print(inspector.get_columns('bgg.ldaOut.top.documents'))

[{'name': 'row_names', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'document', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'topic', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'gamma', 'type': REAL(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}]


In [27]:
#set the table column information to a variable
fields = inspector.get_columns('bgg.ldaOut.top.documents')

In [28]:
#put the information into a dataframe for readability
field_names = pd.DataFrame.from_dict(fields)
field_names

,autoincrement,default,name,nullable,primary_key,type
0,auto,None,row_names,True,0,TEXT
1,auto,None,document,True,0,TEXT
2,auto,None,topic,True,0,INTEGER
3,auto,None,gamma,True,0,REAL


### Collect Data

In many situations, the easiest method of data collection from a database is to get all the rows from a table, then filter the data in Python. However, in some cases (especially when there are millions of rows) it is better to query the database with a specific condition (for example, to get all the rows in a table where the customers live in the state of Maryland) to reduce the load when the data is read in-memory to Jupyter Notebooks. The `pd.read_sql()` function will send a SQL query statement to the database, then return the information collected and put it into a pandas dataframe.

Different database applications may have their own "dialect" of SQL but the general structure is consistent across all styles. Use this **[cheat sheet](http://www.sqltutorial.org/sql-cheat-sheet/)** to learn more SQL statement commands.

In [44]:
#get all the rows from the "test" table in the database
sql = "SELECT * FROM 'bgg.ldaOut.top.documents';"

#the read_sql function takes in the SQL statement for the information requested and the engine (to connect to database)
documents_df = pd.read_sql(sql, engine)
documents_df

,row_names,document,topic,gamma
0,1,The.Lounge..A.Mafia.Game,1,0.203125
1,2,Ultimate.Werewolf..Ultimate.Edition,1,0.200000
2,3,Ultimate.Werewolf..Deluxe.Edition.For.Kickstarter,1,0.200000
3,4,Werewolf,1,0.194805
4,5,Παλέρμο..Το.Μεγάλο.Ξεκαθάρισμα,1,0.190476
5,6,Ultimate.Werewolf..Whitebox.Edition,1,0.187500
6,7,Una.Notte.da.Lupi,1,0.180328
7,8,Mafia..Vendetta,1,0.163934
8,9,The.Werewolves.of.Miller.s.Hollow..The.Pact,1,0.163934
9,10,Ultimate.Werewolf..Deluxe.Edition,1,0.161765


In [45]:
#show the number of rows and columns in each table
documents_df.shape

(288, 4)

In [35]:
#query with conditional
#get all the rows where the student grade is a 76, 77, or 78
sql = "SELECT * FROM 'bgg.ldaOut.top.documents' WHERE topic IN (22);"

topic_df = pd.read_sql(sql, engine)
topic_df

,row_names,document,topic,gamma
0,245,De.Vulgari.Eloquentia,22,0.234375
1,246,Renaissance,22,0.215385
2,247,Princes.of.the.Renaissance,22,0.211268
3,248,Machiavelli,22,0.208333
4,249,Princeps.Machiavelli,22,0.203125
5,250,Venetia,22,0.166667
6,251,Shogun.1,22,0.156250
7,252,Age.of.Renaissance,22,0.152542
8,253,Carcassonne..Wheel.of.Fortune,22,0.140351
9,254,The.Princes.of.Florence,22,0.137931


In [46]:
#reading the data for each table
sql = "SELECT * FROM 'bgg.ldaOut.top.terms';"
terms_df = pd.read_sql(sql, engine)
terms_df

,row_names,topic,term,beta
0,1,1,Bluffing.category,0.183436
1,2,1,Deduction.category,0.164224
2,3,1,Partnerships.mechanic,0.080226
3,4,1,PartyGame.category,0.066343
4,5,1,Murder.Mystery.category,0.048674
5,6,1,RolePlaying.mechanic,0.040120
6,7,1,Voting.mechanic,0.037596
7,8,1,PlayerElimination.mechanic,0.035072
8,9,1,Betting.Wagering.mechanic,0.033108
9,10,1,Spies.SecretAgents.category,0.029743


In [48]:
terms_df.shape

(250, 4)

In [49]:
sql = "SELECT * FROM 'bgg.ldaOut.topics';"
topics_df = pd.read_sql(sql, engine)
topics_df

,row_names,topics(bgg.ldaOut)
0,Die.Macher,8
1,Dragonmaster,5
2,Samurai,22
3,Tal.der.Könige,4
4,Acquire,4
5,Mare.Mediterraneum,22
6,Cathedral,24
7,Lords.of.Creation,9
8,El.Caballero,7
9,Elfenland,25


In [50]:
topics_df.shape

(29229, 2)

In [51]:
sql = "SELECT * FROM 'bgg.topics';"
topics2_df = pd.read_sql(sql, engine)
topics2_df

,row_names,game.id,game.type,details.maxplayers,details.maxplaytime,details.minage,details.minplayers,details.minplaytime,details.name,details.playingtime,...,attributes.t.links.concat.2....,stats.subtype.rpgitem.bayesaverage,stats.subtype.rpgitem.pos,stats.weight.factor,details.playingtime.factor,details.minplayers.factor,details.maxplayers.factor,details.minage.factor,stats.average.factor,topic
0,1,1,boardgame,5.0,240.0,14.0,3.0,240.0,Die Macher,240.0,...,None,NaN,NaN,4,240+,3,5,14,"[7.00,9.83]",Monopoly-inspired
1,2,2,boardgame,4.0,30.0,12.0,3.0,30.0,Dragonmaster,30.0,...,None,NaN,NaN,2,30,3,4,12,"[6.40,7.00)",Traditional Card Games
2,3,3,boardgame,4.0,60.0,10.0,2.0,30.0,Samurai,60.0,...,None,NaN,NaN,3,60,2,4,10,"[7.00,9.83]",Area Control Games
3,4,4,boardgame,4.0,60.0,12.0,2.0,60.0,Tal der Könige,60.0,...,None,NaN,NaN,3,60,2,4,12,"[6.40,7.00)",Corporate Strategy Games
4,5,5,boardgame,6.0,90.0,12.0,3.0,90.0,Acquire,90.0,...,None,NaN,NaN,3,90,3,6,12,"[7.00,9.83]",Corporate Strategy Games
5,6,6,boardgame,6.0,240.0,12.0,2.0,240.0,Mare Mediterraneum,240.0,...,None,NaN,NaN,3,240+,2,6,12,"[6.40,7.00)",Area Control Games
6,7,7,boardgame,2.0,20.0,8.0,2.0,20.0,Cathedral,15.0,...,None,NaN,NaN,2,15,2,2,8,"[6.40,7.00)",Abstract Strategy Games
7,8,8,boardgame,5.0,120.0,12.0,2.0,120.0,Lords of Creation,120.0,...,None,NaN,NaN,2,120,2,5,12,"[5.86,6.40)",Dexterity Games
8,9,9,boardgame,4.0,90.0,13.0,2.0,90.0,El Caballero,90.0,...,None,NaN,NaN,3,90,2,4,13,"[6.40,7.00)",Modern Wargames
9,10,10,boardgame,6.0,60.0,10.0,2.0,60.0,Elfenland,60.0,...,None,NaN,NaN,2,60,2,6,10,"[6.40,7.00)",Modern Card Games


In [52]:
topics2_df.shape

(29313, 74)